In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
df_movies = pd.read_json('../raw/movie.json')
df_movies.info()

In [ ]:
print(df_movies.head(1))

In [ ]:
def write_to_json(path, list_data):
    with open(path, 'w') as file:
        json.dump(list_data, file, indent=2)
def checkStringEmpty(value):
    if value == "" or value is None:
        return True
    return False
def getLanguage(value):
    if value is None or len(value) == 0:
        return None
    return value[0]['name']
def getImageUrl(value):
    if checkStringEmpty(value):
        return None
    return 'https://image.tmdb.org/t/p/original' + value

In [ ]:
List_movie = []
list_company = []
# loop through the rows using iterrows()
for index, row in df_movies.iterrows():

    genres = [ genre['id'] for genre in row['genres']]

    companies = [ company['id'] for company in row['production_companies']]

    list_company.extend(companies)

    movie = {
        "id": row['id'],
        "imdb_id": row['imdb_id'],
        "title": row['title'],
        "genres": genres,
        "original_language": row['original_language'],
        "original_title": row['original_title'],
        "overview": row['overview'],
        "poster_path": getImageUrl(row['poster_path']),
        "backdrop_path": getImageUrl(row['backdrop_path']),
        "release_date": row['release_date'],
        "revenue": row['revenue'],
        "budget": row['budget'],
        "spoken_languages": row['spoken_languages'],
        "production_countries": row['production_countries'],
        "production_companies": companies,
        "status": row['status'],
        "video": row['video'],
        "vote_average": row['vote_average'],
        "vote_count": row['vote_count']
    }

    List_movie.append(movie)

write_to_json('./data/movie.json', List_movie)

set_companies = set(list_company)
write_to_json('./data/company.json', list(set_companies))
    

        